## Multi-class classification using ERNIE

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
pip install accelerate -U

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from transformers import TrainerCallback
import os
import shutil
import re
import time
from pathlib import Path
# [1e-05, 0.01, 2, 32, 64]

def main_model(file_name, ext, type):

    path_type = "Balanced" if type == 1 else "Unbalanced"

    current_file_path = Path(__file__).parent
    path_to_project = current_file_path.parents[1]

    df = pd.read_excel(f"{path_to_project}/Data/Datasets/{path_type}/{file_name}.{ext}")

    results_dir = f"{path_to_project}/Models/ERNIE/Output/{path_type}/{file_name}"
    dump_dir = results_dir+"/Dump"

    if os.path.isdir(results_dir):
        shutil.rmtree(results_dir)

    os.mkdir(results_dir)
    os.mkdir(dump_dir)

    df = df[df['review'].notna() & (df['review'] != '')]
    # Select the text and label columns
    df['review'] = df['review'].str.replace('[^\x20-\x7E]', '', regex=True)
    X = df['review'].values
    y = df['label'].values

    X_train_CV, X_test_full, y_train_CV, y_test_full = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


    # Encode the labels to a numeric format
    label_encoder = LabelEncoder()
    y_train_CV_encoded = label_encoder.fit_transform(y_train_CV)
    y_test_full_encoded = label_encoder.transform(y_test_full)

    # Initialize the tokenizer for RoBERTa
    tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-base-en")

    # Tokenization function
    def tokenize_function(texts):
        return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

    loss_logging_callback = LossLoggingCallback()

    # Stratified K-Fold Cross-Validation
    n_splits = 5
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Variables to accumulate scores
    best_accuracy = 0
    best_model = None
    accuracy_scores = []
    metrics_df = pd.DataFrame()


    for fold, (train_index, val_index) in enumerate(kf.split(X_train_CV, y_train_CV_encoded)):
        print(f"Fold {fold+1}/{n_splits}")
        start_time = time.time()
        # Split the data
        X_train, X_val = X_train_CV[train_index], X_train_CV[val_index]
        y_train, y_val = y_train_CV_encoded[train_index], y_train_CV_encoded[val_index]


        # Tokenize the data
        train_encodings = tokenize_function(X_train.tolist())
        val_encodings = tokenize_function(X_val.tolist())

        # Create dataset objects
        train_dataset = ReviewDataset(train_encodings, y_train)
        val_dataset = ReviewDataset(val_encodings, y_val)

        # Initialize the model for each fold
        model = AutoModelForSequenceClassification.from_pretrained("nghuyong/ernie-2.0-base-en", num_labels=len(label_encoder.classes_))

        # Define training arguments for each fold, adjust hyperparameters as needed
        training_args = TrainingArguments(
            output_dir=f"{dump_dir}/res",
            num_train_epochs=5,
            per_device_train_batch_size=10,
            per_device_eval_batch_size=49,
            warmup_steps=500,
            weight_decay=0.07996589256970411,
            logging_dir=f"{dump_dir}/logs",
            logging_strategy="epoch",
            evaluation_strategy="epoch",
            learning_rate=3.592687488204789e-05,
            max_grad_norm=1.0,
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            save_strategy="epoch",
            save_total_limit=2,
            lr_scheduler_type='linear'
        )

        # Initialize Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=lambda p: {"accuracy": accuracy_score(p.predictions.argmax(-1), p.label_ids)},
            callbacks=[loss_logging_callback]
        )

        # Train
        trainer.train()

        loss_logging_callback.save_logs_to_excel(f"{results_dir}/fold_loss.xlsx")

        # Evaluate
        results = trainer.evaluate()
        accuracy_scores.append(results['eval_accuracy'])

        if results['eval_accuracy'] > best_accuracy:
            best_accuracy = results['eval_accuracy']
            best_model = model  # Assign the best model

        # Get predictions and true labels
        predictions = trainer.predict(val_dataset)
        pred_labels = np.argmax(predictions.predictions, axis=-1)
        true_labels = y_val

        # Calculate accuracy
        accuracy = accuracy_score(true_labels, pred_labels)
        label_names = label_encoder.inverse_transform(range(len(label_encoder.classes_)))

        # Calculate precision, recall, and F1-score
        report_dict = classification_report(true_labels, pred_labels, output_dict=True, zero_division=0, target_names=label_names)
        # avg_metrics = report_dict['weighted avg']  # Use 'macro avg' or 'weighted avg' based on your preference
        end_time = time.time()
        # Append the metrics for this fold to the DataFrame
        metrics_df = metrics_df.append({
            ('Fold', ''): fold + 1,
            ('Accuracy', ''): accuracy,
            ('Train Time', ''): str(end_time - start_time)+" s",
            ('Bug Report', 'P'): report_dict['bug report']['precision'],
            ('Bug Report', 'R'): report_dict['bug report']['recall'],
            ('Bug Report', 'F1'): report_dict['bug report']['f1-score'],
            ('Feature Request', 'P'): report_dict['feature request']['precision'],
            ('Feature Request', 'R'): report_dict['feature request']['recall'],
            ('Feature Request', 'F1'): report_dict['feature request']['f1-score'],
            ('Rating', 'P'): report_dict['rating']['precision'],
            ('Rating', 'R'): report_dict['rating']['recall'],
            ('Rating', 'F1'): report_dict['rating']['f1-score'],
            ('User Experience', 'P'): report_dict['user experience']['precision'],
            ('User Experience', 'R'): report_dict['user experience']['recall'],
            ('User Experience', 'F1'): report_dict['user experience']['f1-score']
        }, ignore_index=True)

    # Save the DataFrame to a CSV file after completing all folds
    metrics_df.columns = pd.MultiIndex.from_tuples([(c,) if isinstance(c, str) else c for c in metrics_df.columns])
    metrics_df.to_excel(f"{results_dir}/fold_metrics.xlsx", index=True)

    # Evaluate the best model on the test set
    test_encodings = tokenize_function(X_test_full.tolist())
    test_dataset = ReviewDataset(test_encodings, y_test_full_encoded)
    test_trainer = Trainer(model=best_model)
    test_results = test_trainer.predict(test_dataset)
    test_predictions = np.argmax(test_results.predictions, axis=-1)
    test_accuracy = accuracy_score(y_test_full_encoded, test_predictions)

    label_names_full = label_encoder.inverse_transform(range(len(label_encoder.classes_)))

    # Calculate precision, recall, and F1-score
    report_dict_full = classification_report(y_test_full_encoded, test_predictions, output_dict=True, zero_division=0, target_names=label_names_full)
    print(report_dict_full)
    full_metrics_df = pd.DataFrame()

    full_metrics_df = full_metrics_df.append({
            ('Accuracy', ''): test_accuracy,
            ('Bug Report', 'P'): report_dict_full['bug report']['precision'],
            ('Bug Report', 'R'): report_dict_full['bug report']['recall'],
            ('Bug Report', 'F1'): report_dict_full['bug report']['f1-score'],
            ('Feature Request', 'P'): report_dict_full['feature request']['precision'],
            ('Feature Request', 'R'): report_dict_full['feature request']['recall'],
            ('Feature Request', 'F1'): report_dict_full['feature request']['f1-score'],
            ('Rating', 'P'): report_dict_full['rating']['precision'],
            ('Rating', 'R'): report_dict_full['rating']['recall'],
            ('Rating', 'F1'): report_dict_full['rating']['f1-score'],
            ('User Experience', 'P'): report_dict_full['user experience']['precision'],
            ('User Experience', 'R'): report_dict_full['user experience']['recall'],
            ('User Experience', 'F1'): report_dict_full['user experience']['f1-score']
        }, ignore_index=True)

    full_metrics_df.columns = pd.MultiIndex.from_tuples([(c,) if isinstance(c, str) else c for c in full_metrics_df.columns])
    full_metrics_df.to_excel(f"{results_dir}/metrics_results_full_test.xlsx", index=True)

    print(f"Test Accuracy: {test_accuracy}")

    # Generate and print the classification report
    print(classification_report(y_test_full_encoded, test_predictions, target_names=label_encoder.classes_, zero_division=0))

    shutil.rmtree(dump_dir)

# Custom dataset class
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

class LossLoggingCallback(TrainerCallback):
    """A custom callback to log training and validation loss."""
    def __init__(self):
        super().__init__()
        self.log_history = []
        self.log_train_loss_history = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        # This method captures both training and evaluation logs, so it's more general than on_epoch_end
        if logs is not None:
            # Capture both training and evaluation steps
            if 'loss' in logs:  # Indicates a training step
                self.log_train_loss_history.append({
                    'epoch': state.epoch,
                    'training_loss': logs.get('loss'),
                })
            elif 'eval_loss' in logs:  # Indicates an evaluation step
                # Make sure to capture the last training loss as well
                last_training_loss = self.log_train_loss_history[-1]['training_loss'] if self.log_train_loss_history else None
                self.log_history.append({
                    'epoch': state.epoch,
                    'training_loss': last_training_loss,  # Include last known training loss for reference
                    'validation_loss': logs.get('eval_loss'),
                    'eval_runtime':logs.get('eval_runtime')
                })

    def save_logs_to_excel(self, file_name):
        """Save the recorded logs to a Excel file."""
        pd.DataFrame(self.log_history).to_excel(file_name, index=False)

__file__ = "/content/drive/MyDrive/FinalProject/Models/ERNIE/ERNIE.ipynb"
current_file_path = Path(__file__).parent
path_to_project = current_file_path.parents[1]

directory_path_multi = path_to_project / 'Data' / 'Datasets' / 'Balanced'

files_multi = [(file.name, file.stat().st_size)
               for file in directory_path_multi.iterdir()
               if file.is_file() and not file.name.startswith('.')]

files_multi.sort(key=lambda x: x[1])





Running the model on each dataset separately due to storage constraints

In [4]:
print(f"Now doing: {files_multi[1][0].split('.')[0]}")
main_model(files_multi[1][0].split('.')[0], files_multi[1][0].split('.')[1], 1)
print(f"Now doing: {files_multi[2][0].split('.')[0]}")
main_model(files_multi[2][0].split('.')[0], files_multi[2][0].split('.')[1], 1)

Now doing: dataset_balanced_4000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.872000,0.658779,0.769531
2,0.530700,0.459666,0.850781
3,0.263300,0.474178,0.888281
4,0.114500,0.510199,0.903906
5,0.048600,0.523981,0.910937


<ipython-input-3-cb515e3ed427>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 0.9026128266033254, 'recall': 0.95, 'f1-score': 0.9257003654080389, 'support': 400}, 'feature request': {'precision': 0.9007444168734491, 'recall': 0.9075, 'f1-score': 0.904109589041096, 'support': 400}, 'rating': {'precision': 0.902200488997555, 'recall': 0.9225, 'f1-score': 0.9122373300370828, 'support': 400}, 'user experience': {'precision': 0.8991825613079019, 'recall': 0.825, 'f1-score': 0.8604954367666231, 'support': 400}, 'accuracy': 0.90125, 'macro avg': {'precision': 0.9011850734455578, 'recall': 0.9012499999999999, 'f1-score': 0.9006356803132102, 'support': 1600}, 'weighted avg': {'precision': 0.9011850734455578, 'recall': 0.90125, 'f1-score': 0.9006356803132103, 'support': 1600}}
Test Accuracy: 0.90125
                 precision    recall  f1-score   support

     bug report       0.90      0.95      0.93       400
feature request       0.90      0.91      0.90       400
         rating       0.90      0.92      0.91       400
user experience    

<ipython-input-3-cb515e3ed427>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


In [5]:
print(f"Now doing: {files_multi[3][0].split('.')[0]}")
main_model(files_multi[3][0].split('.')[0], files_multi[3][0].split('.')[1], 1)
print(f"Now doing: {files_multi[4][0].split('.')[0]}")
main_model(files_multi[4][0].split('.')[0], files_multi[4][0].split('.')[1], 1)

Now doing: dataset_gpt_balanced_4000
Fold 1/5


Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.188800,0.617782,0.790625
2,0.361900,0.297733,0.935937
3,0.262900,0.250673,0.951562
4,0.215400,0.213353,0.953125
5,0.152300,0.219592,0.950000


<ipython-input-3-cb515e3ed427>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.682600,0.213205,0.962471
2,0.165600,0.207592,0.963253
3,0.125400,0.171090,0.971853
4,0.113300,0.144443,0.970289
5,0.083200,0.142981,0.971071


<ipython-input-3-cb515e3ed427>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 0.9922680412371134, 'recall': 0.9625, 'f1-score': 0.9771573604060914, 'support': 400}, 'feature request': {'precision': 1.0, 'recall': 0.98, 'f1-score': 0.98989898989899, 'support': 400}, 'rating': {'precision': 0.9799498746867168, 'recall': 0.9775, 'f1-score': 0.9787234042553192, 'support': 400}, 'user experience': {'precision': 0.9477434679334917, 'recall': 0.9975, 'f1-score': 0.971985383678441, 'support': 400}, 'accuracy': 0.979375, 'macro avg': {'precision': 0.9799903459643304, 'recall': 0.979375, 'f1-score': 0.9794412845597105, 'support': 1600}, 'weighted avg': {'precision': 0.9799903459643307, 'recall': 0.979375, 'f1-score': 0.9794412845597102, 'support': 1600}}
Test Accuracy: 0.979375
                 precision    recall  f1-score   support

     bug report       0.99      0.96      0.98       400
feature request       1.00      0.98      0.99       400
         rating       0.98      0.98      0.98       400
user experience       0.95      1.00     

<ipython-input-3-cb515e3ed427>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


In [7]:
print(f"Now doing: {files_multi[5][0].split('.')[0]}")
main_model(files_multi[5][0].split('.')[0], files_multi[5][0].split('.')[1], 1)

Now doing: dataset_gpt_balanced_20000
Fold 1/5


Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.326300,0.105641,0.982187
2,0.069600,0.072500,0.988125
3,0.059200,0.070755,0.988437
4,0.052600,0.065328,0.987812
5,0.039600,0.067753,0.988125


<ipython-input-3-cb515e3ed427>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.330900,0.086606,0.985933
2,0.075300,0.071269,0.986871
3,0.066500,0.076213,0.987809
4,0.052100,0.066323,0.988434
5,0.040100,0.075243,0.987496


<ipython-input-3-cb515e3ed427>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 0.9989878542510121, 'recall': 0.987, 'f1-score': 0.9929577464788732, 'support': 1000}, 'feature request': {'precision': 1.0, 'recall': 0.991, 'f1-score': 0.9954796584630838, 'support': 1000}, 'rating': {'precision': 0.992, 'recall': 0.992, 'f1-score': 0.992, 'support': 1000}, 'user experience': {'precision': 0.9774730656219393, 'recall': 0.998, 'f1-score': 0.987629886194953, 'support': 1000}, 'accuracy': 0.992, 'macro avg': {'precision': 0.9921152299682379, 'recall': 0.992, 'f1-score': 0.9920168227842275, 'support': 4000}, 'weighted avg': {'precision': 0.9921152299682379, 'recall': 0.992, 'f1-score': 0.9920168227842275, 'support': 4000}}
Test Accuracy: 0.992
                 precision    recall  f1-score   support

     bug report       1.00      0.99      0.99      1000
feature request       1.00      0.99      1.00      1000
         rating       0.99      0.99      0.99      1000
user experience       0.98      1.00      0.99      1000

       accuracy  

<ipython-input-3-cb515e3ed427>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


In [8]:
print(f"Now doing: {files_multi[6][0].split('.')[0]}")
main_model(files_multi[6][0].split('.')[0], files_multi[6][0].split('.')[1], 1)

Now doing: dataset_gpt_balanced_32000
Fold 1/5


Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.203500,0.071254,0.990430
2,0.053700,0.053818,0.991406
3,0.045300,0.040816,0.992188
4,0.040900,0.051745,0.990430
5,0.036100,0.043250,0.992969


<ipython-input-3-cb515e3ed427>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.213900,0.054800,0.991600
2,0.055800,0.044293,0.992577
3,0.041500,0.042649,0.992772
4,0.040300,0.040241,0.993163
5,0.033800,0.042012,0.993163


<ipython-input-3-cb515e3ed427>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 1.0, 'recall': 0.9925, 'f1-score': 0.9962358845671266, 'support': 1600}, 'feature request': {'precision': 0.9993710691823899, 'recall': 0.993125, 'f1-score': 0.9962382445141066, 'support': 1600}, 'rating': {'precision': 0.9931292941911305, 'recall': 0.99375, 'f1-score': 0.9934395501405809, 'support': 1600}, 'user experience': {'precision': 0.987045033929673, 'recall': 1.0, 'f1-score': 0.9934802856255821, 'support': 1600}, 'accuracy': 0.99484375, 'macro avg': {'precision': 0.9948863493257983, 'recall': 0.99484375, 'f1-score': 0.9948484912118492, 'support': 6400}, 'weighted avg': {'precision': 0.9948863493257984, 'recall': 0.99484375, 'f1-score': 0.994848491211849, 'support': 6400}}
Test Accuracy: 0.99484375
                 precision    recall  f1-score   support

     bug report       1.00      0.99      1.00      1600
feature request       1.00      0.99      1.00      1600
         rating       0.99      0.99      0.99      1600
user experience       0.99

<ipython-input-3-cb515e3ed427>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


In [9]:
directory_path_unbalanced = path_to_project / 'Data' / 'Datasets' / 'Unbalanced'

files_unbalanced = [(file.name, file.stat().st_size)
               for file in directory_path_unbalanced.iterdir()
               if file.is_file() and not file.name.startswith('.')]

files_unbalanced.sort(key=lambda x: x[1])

In [10]:
print(f"Now doing: {files_unbalanced[0][0].split('.')[0]}")
main_model(files_unbalanced[0][0].split('.')[0], files_unbalanced[0][0].split('.')[1], 2)
print(f"Now doing: {files_unbalanced[1][0].split('.')[0]}")
main_model(files_unbalanced[1][0].split('.')[0], files_unbalanced[1][0].split('.')[1], 2)

Now doing: dataset_unbalanced_4000
Fold 1/5


Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.968400,0.639955,0.776563
2,0.631300,0.550206,0.796875
3,0.389400,0.730154,0.800000
4,0.215900,0.719427,0.842187
5,0.069500,0.726607,0.853125


<ipython-input-3-cb515e3ed427>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.033700,0.503624,0.835938
2,0.132200,0.265148,0.956250
3,0.067600,0.149970,0.975000
4,0.029600,0.136568,0.982812
5,0.020300,0.132609,0.982812


<ipython-input-3-cb515e3ed427>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 0.98, 'recall': 0.98, 'f1-score': 0.98, 'support': 100}, 'feature request': {'precision': 0.9868421052631579, 'recall': 1.0, 'f1-score': 0.9933774834437086, 'support': 150}, 'rating': {'precision': 1.0, 'recall': 0.98, 'f1-score': 0.98989898989899, 'support': 250}, 'user experience': {'precision': 0.9801980198019802, 'recall': 0.99, 'f1-score': 0.9850746268656716, 'support': 300}, 'accuracy': 0.9875, 'macro avg': {'precision': 0.9867600312662845, 'recall': 0.9875, 'f1-score': 0.9870877750520926, 'support': 800}, 'weighted avg': {'precision': 0.9876071521625847, 'recall': 0.9875, 'f1-score': 0.9875046975637565, 'support': 800}}
Test Accuracy: 0.9875
                 precision    recall  f1-score   support

     bug report       0.98      0.98      0.98       100
feature request       0.99      1.00      0.99       150
         rating       1.00      0.98      0.99       250
user experience       0.98      0.99      0.99       300

       accuracy            

<ipython-input-3-cb515e3ed427>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


In [11]:
print(f"Now doing: {files_unbalanced[2][0].split('.')[0]}")
main_model(files_unbalanced[2][0].split('.')[0], files_unbalanced[2][0].split('.')[1], 2)

Now doing: dataset_gpt_unbalanced_8000
Fold 1/5


Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.619200,0.132124,0.975000
2,0.075600,0.088967,0.986719
3,0.047400,0.056272,0.992188
4,0.039900,0.063727,0.990625
5,0.025600,0.052724,0.990625


<ipython-input-3-cb515e3ed427>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.576600,0.119268,0.982799
2,0.059800,0.046429,0.993745
3,0.041200,0.070789,0.990618
4,0.032400,0.071396,0.989054
5,0.021300,0.072983,0.988272


<ipython-input-3-cb515e3ed427>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 1.0, 'recall': 0.98, 'f1-score': 0.98989898989899, 'support': 200}, 'feature request': {'precision': 0.9900332225913622, 'recall': 0.9933333333333333, 'f1-score': 0.9916805324459235, 'support': 300}, 'rating': {'precision': 1.0, 'recall': 0.988, 'f1-score': 0.993963782696177, 'support': 500}, 'user experience': {'precision': 0.9819376026272578, 'recall': 0.9966666666666667, 'f1-score': 0.9892473118279571, 'support': 600}, 'accuracy': 0.99125, 'macro avg': {'precision': 0.992992706304655, 'recall': 0.9894999999999999, 'f1-score': 0.9911976542172619, 'support': 1600}, 'weighted avg': {'precision': 0.9913578302211021, 'recall': 0.99125, 'f1-score': 0.9912588975990236, 'support': 1600}}
Test Accuracy: 0.99125
                 precision    recall  f1-score   support

     bug report       1.00      0.98      0.99       200
feature request       0.99      0.99      0.99       300
         rating       1.00      0.99      0.99       500
user experience       0.98 

<ipython-input-3-cb515e3ed427>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


In [13]:
print(f"Now doing: {files_unbalanced[3][0].split('.')[0]}")
main_model(files_unbalanced[3][0].split('.')[0], files_unbalanced[3][0].split('.')[1], 2)

Now doing: dataset_gpt_unbalanced_16000
Fold 1/5


Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.360800,0.065576,0.989453
2,0.046700,0.053249,0.991797
3,0.041600,0.051581,0.992188
4,0.031700,0.063536,0.991406
5,0.024500,0.062026,0.991797


<ipython-input-3-cb515e3ed427>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.326300,0.041155,0.993357
2,0.053500,0.039599,0.994920
3,0.042400,0.032715,0.995701
4,0.034900,0.028007,0.995701
5,0.017000,0.039934,0.994920


<ipython-input-3-cb515e3ed427>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 1.0, 'recall': 0.985, 'f1-score': 0.9924433249370278, 'support': 400}, 'feature request': {'precision': 1.0, 'recall': 0.9966666666666667, 'f1-score': 0.9983305509181971, 'support': 600}, 'rating': {'precision': 0.9979959919839679, 'recall': 0.996, 'f1-score': 0.9969969969969971, 'support': 1000}, 'user experience': {'precision': 0.9917355371900827, 'recall': 1.0, 'f1-score': 0.995850622406639, 'support': 1200}, 'accuracy': 0.99625, 'macro avg': {'precision': 0.9974328822935126, 'recall': 0.9944166666666667, 'f1-score': 0.9959053738147152, 'support': 3200}, 'weighted avg': {'precision': 0.9962745739412711, 'recall': 0.99625, 'f1-score': 0.9962479388783416, 'support': 3200}}
Test Accuracy: 0.99625
                 precision    recall  f1-score   support

     bug report       1.00      0.98      0.99       400
feature request       1.00      1.00      1.00       600
         rating       1.00      1.00      1.00      1000
user experience       0.99      1.00

<ipython-input-3-cb515e3ed427>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


## Multi-label classification using ERNIE

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from transformers import TrainerCallback
import os
import shutil
import re
import time
from pathlib import Path

def multi_main_model(file_name, ext):

    current_file_path = Path(__file__).parent

    path_to_project = current_file_path.parents[1]

    df = pd.read_csv(f"{path_to_project}/Data/Datasets/Multi-label/{file_name}.{ext}")

    results_dir = f"{path_to_project}/Models/ERNIE/Output/Multi-label/{file_name}"
    dump_dir = results_dir+"/Dump"

    if os.path.isdir(results_dir):
        shutil.rmtree(results_dir)

    os.mkdir(results_dir)
    os.mkdir(dump_dir)

    df = df[df['review'].notna() & (df['review'] != '')]
    df['review'] = df['review'].str.replace('[^\x20-\x7E]', '', regex=True)

    X = df['review'].values
    y = df[['bug report', 'user experience', 'rating', 'feature request']].values

    X_train_CV, X_test_full, y_train_CV, y_test_full = train_test_split(X, y, test_size=0.2, random_state=42)

    tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-base-en")

    def tokenize_function(examples):
        return tokenizer(examples, padding="max_length", truncation=True, max_length=128)

    loss_logging_callback = LossLoggingCallback()

    # K-Fold Cross-Validation
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Variables to accumulate scores
    best_f1 = 0
    best_model = None
    metrics_df = pd.DataFrame()

    for fold, (train_index, val_index) in enumerate(kf.split(X_train_CV, y_train_CV)):
        print(f"Fold {fold+1}/{n_splits}")
        start_time = time.time()

        X_train, X_val = X_train_CV[train_index], X_train_CV[val_index]
        y_train, y_val = y_train_CV[train_index], y_train_CV[val_index]

        train_encodings = tokenize_function(X_train.tolist())
        val_encodings = tokenize_function(X_val.tolist())

        train_dataset = MultiLabelDataset(train_encodings, y_train)
        val_dataset = MultiLabelDataset(val_encodings, y_val)

        model = AutoModelForSequenceClassification.from_pretrained("nghuyong/ernie-2.0-base-en", num_labels=4, problem_type="multi_label_classification")

        training_args = TrainingArguments(
            output_dir=f"{dump_dir}/res",
            num_train_epochs=5,
            per_device_train_batch_size=10,
            per_device_eval_batch_size=49,
            warmup_steps=500,
            weight_decay=0.07,
            logging_dir=f"{dump_dir}/logs",
            logging_strategy="epoch",
            evaluation_strategy="epoch",
            learning_rate=3e-5,
            max_grad_norm=1.0,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            save_strategy="epoch",
            save_total_limit=2,
            lr_scheduler_type='linear'
        )

        def compute_metrics(p):
            predictions, labels = p
            predictions = torch.sigmoid(torch.tensor(predictions)).numpy()
            threshold = 0.5
            predictions = (predictions > threshold).astype(int)
            precision = precision_score(labels, predictions, average='micro')
            recall = recall_score(labels, predictions, average='micro')
            f1 = f1_score(labels, predictions, average='micro')
            return {'precision': precision, 'recall': recall, 'f1': f1}


        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics,
            callbacks=[loss_logging_callback]
        )

        trainer.train()

        loss_logging_callback.save_logs_to_excel(f"{results_dir}/fold_loss.xlsx")

        results = trainer.evaluate()

        if results['eval_f1'] > best_f1:
            best_f1 = results['eval_f1']
            best_model = model


        predictions = trainer.predict(val_dataset)
        pred_probs = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
        threshold = 0.5
        binary_predictions = (pred_probs > threshold).astype(int)

        # True labels
        true_labels = predictions.label_ids
        f1 = f1_score(true_labels, binary_predictions, average='micro')

        report_dict = classification_report(true_labels, binary_predictions, output_dict=True, zero_division=0, target_names=['bug report', 'user experience', 'rating', 'feature request'])
        # avg_metrics = report_dict['weighted avg']  # Use 'macro avg' or 'weighted avg' based on your preference
        end_time = time.time()
        # Append the metrics for this fold to the DataFrame
        metrics_df = metrics_df.append({
            ('Fold', ''): fold + 1,
            ('F1-Score', ''): f1,
            ('Train Time', ''): str(end_time - start_time)+" s",
            ('Bug Report', 'P'): report_dict['bug report']['precision'],
            ('Bug Report', 'R'): report_dict['bug report']['recall'],
            ('Bug Report', 'F1'): report_dict['bug report']['f1-score'],
            ('Feature Request', 'P'): report_dict['feature request']['precision'],
            ('Feature Request', 'R'): report_dict['feature request']['recall'],
            ('Feature Request', 'F1'): report_dict['feature request']['f1-score'],
            ('Rating', 'P'): report_dict['rating']['precision'],
            ('Rating', 'R'): report_dict['rating']['recall'],
            ('Rating', 'F1'): report_dict['rating']['f1-score'],
            ('User Experience', 'P'): report_dict['user experience']['precision'],
            ('User Experience', 'R'): report_dict['user experience']['recall'],
            ('User Experience', 'F1'): report_dict['user experience']['f1-score']
        }, ignore_index=True)

    metrics_df.columns = pd.MultiIndex.from_tuples([(c,) if isinstance(c, str) else c for c in metrics_df.columns])
    metrics_df.to_excel(f"{results_dir}/fold_metrics.xlsx", index=True)

    test_encodings = tokenize_function(X_test_full.tolist())
    test_dataset = MultiLabelDataset(test_encodings, y_test_full)
    test_trainer = Trainer(model=best_model)
    test_predictions = test_trainer.predict(test_dataset)
    test_pred_probs = torch.sigmoid(torch.tensor(test_predictions.predictions)).numpy()
    threshold = 0.5
    test_binary_predictions = (test_pred_probs > threshold).astype(int)

    test_true_labels = test_predictions.label_ids
    test_f1 = f1_score(test_true_labels, test_binary_predictions, average='micro')

    test_report_dict = classification_report(test_true_labels, test_binary_predictions, output_dict=True, zero_division=0, target_names=['bug report', 'user experience', 'rating', 'feature request'])
    # avg_metrics = report_dict['weighted avg']  # Use 'macro avg' or 'weighted avg' based on your preference
    # Append the metrics for this fold to the DataFrame
    test_metrics_df = pd.DataFrame()

    test_metrics_df = test_metrics_df.append({
            ('F1', ''): test_f1,
            ('Bug Report', 'P'): test_report_dict['bug report']['precision'],
            ('Bug Report', 'R'): test_report_dict['bug report']['recall'],
            ('Bug Report', 'F1'): test_report_dict['bug report']['f1-score'],
            ('Feature Request', 'P'): test_report_dict['feature request']['precision'],
            ('Feature Request', 'R'): test_report_dict['feature request']['recall'],
            ('Feature Request', 'F1'): test_report_dict['feature request']['f1-score'],
            ('Rating', 'P'): test_report_dict['rating']['precision'],
            ('Rating', 'R'): test_report_dict['rating']['recall'],
            ('Rating', 'F1'): test_report_dict['rating']['f1-score'],
            ('User Experience', 'P'): test_report_dict['user experience']['precision'],
            ('User Experience', 'R'): test_report_dict['user experience']['recall'],
            ('User Experience', 'F1'): test_report_dict['user experience']['f1-score']
        }, ignore_index=True)

    test_metrics_df.columns = pd.MultiIndex.from_tuples([(c,) if isinstance(c, str) else c for c in test_metrics_df.columns])
    test_metrics_df.to_excel(f"{results_dir}/metrics_results_full_test.xlsx", index=True)

    print(f"Test F1: {test_f1}")

    # Generate and print the classification report
    print(test_report_dict)

    shutil.rmtree(dump_dir)

class MultiLabelDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float32)  # Ensure float32 for BCEWithLogitsLoss
        return item

    def __len__(self):
        return len(self.labels)
class LossLoggingCallback(TrainerCallback):
    """A custom callback to log training and validation loss."""
    def __init__(self):
        super().__init__()
        self.log_history = []
        self.log_train_loss_history = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        # This method captures both training and evaluation logs, so it's more general than on_epoch_end
        if logs is not None:
            # Capture both training and evaluation steps
            if 'loss' in logs:  # Indicates a training step
                self.log_train_loss_history.append({
                    'epoch': state.epoch,
                    'training_loss': logs.get('loss'),
                })
            elif 'eval_loss' in logs:  # Indicates an evaluation step
                # Make sure to capture the last training loss as well
                last_training_loss = self.log_train_loss_history[-1]['training_loss'] if self.log_train_loss_history else None
                self.log_history.append({
                    'epoch': state.epoch,
                    'training_loss': last_training_loss,  # Include last known training loss for reference
                    'validation_loss': logs.get('eval_loss'),
                    'eval_runtime':logs.get('eval_runtime')
                })

    def save_logs_to_excel(self, file_name):
        """Save the recorded logs to a Excel file."""
        pd.DataFrame(self.log_history).to_excel(file_name, index=False)

__file__ = "/content/drive/MyDrive/FinalProject/Models/ERNIE/ERNIE.ipynb"
current_file_path = Path(__file__).parent
path_to_project = current_file_path.parents[1]

directory_path_multi_label = path_to_project / 'Data' / 'Datasets' / 'Multi-label'

files_multi_label = [(file.name, file.stat().st_size)
               for file in directory_path_multi_label.iterdir()
               if file.is_file() and not file.name.startswith('.')]

files_multi_label.sort(key=lambda x: x[1])

In [5]:
print(f"Now doing: {files_multi_label[1][0].split('.')[0]}")
multi_main_model(files_multi_label[1][0].split('.')[0], files_multi_label[1][0].split('.')[1])

Now doing: dataset_gpt_multi_label_4000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.601600,0.505865,0.789855,0.626952,0.699038
2,0.359300,0.282151,0.943171,0.845522,0.891681
3,0.230200,0.231890,0.993184,0.838127,0.909091
4,0.160600,0.191561,0.981115,0.896467,0.936883
5,0.110000,0.180542,0.978667,0.904684,0.940222


<ipython-input-4-b6cc06e28271>:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


<ipython-input-4-b6cc06e28271>:170: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_metrics_df = test_metrics_df.append({


Test F1: 0.932905067808708
{'bug report': {'precision': 0.9889705882352942, 'recall': 0.9340277777777778, 'f1-score': 0.9607142857142857, 'support': 288}, 'user experience': {'precision': 0.980565371024735, 'recall': 0.9422750424448217, 'f1-score': 0.961038961038961, 'support': 589}, 'rating': {'precision': 0.9941520467836257, 'recall': 0.6614785992217899, 'f1-score': 0.794392523364486, 'support': 257}, 'feature request': {'precision': 0.9690402476780186, 'recall': 0.9315476190476191, 'f1-score': 0.9499241274658573, 'support': 336}, 'micro avg': {'precision': 0.9812312312312312, 'recall': 0.8891156462585034, 'f1-score': 0.932905067808708, 'support': 1470}, 'macro avg': {'precision': 0.9831820634304184, 'recall': 0.8673322596230021, 'f1-score': 0.9165174743958975, 'support': 1470}, 'weighted avg': {'precision': 0.9819531511486664, 'recall': 0.8891156462585034, 'f1-score': 0.9293000324291586, 'support': 1470}, 'samples avg': {'precision': 0.8754166666666666, 'recall': 0.8228125, 'f1-scor

In [6]:
print(f"Now doing: {files_multi_label[2][0].split('.')[0]}")
multi_main_model(files_multi_label[2][0].split('.')[0], files_multi_label[2][0].split('.')[1])
print(f"Now doing: {files_multi_label[3][0].split('.')[0]}")
multi_main_model(files_multi_label[3][0].split('.')[0], files_multi_label[3][0].split('.')[1])

Now doing: dataset_gpt_multi_label_8000
Fold 1/5


Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.443400,0.212085,0.998404,0.842838,0.914049
2,0.186000,0.177014,0.993830,0.867984,0.926654
3,0.135300,0.116267,0.992802,0.929053,0.959870
4,0.097500,0.115741,0.986275,0.935788,0.960369
5,0.074700,0.117154,0.984899,0.937135,0.960423


<ipython-input-4-b6cc06e28271>:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.284000,0.101874,1.000000,0.937746,0.967873
2,0.091000,0.079703,0.995867,0.949304,0.972028
3,0.072400,0.075381,0.992886,0.953244,0.972661
4,0.059100,0.067742,0.994295,0.961387,0.977564
5,0.035400,0.063873,0.992453,0.967166,0.979646


<ipython-input-4-b6cc06e28271>:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


<ipython-input-4-b6cc06e28271>:170: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_metrics_df = test_metrics_df.append({


Test F1: 0.9825798867158276
{'bug report': {'precision': 0.9977324263038548, 'recall': 0.9745293466223699, 'f1-score': 0.9859943977591036, 'support': 903}, 'user experience': {'precision': 0.9950641658440277, 'recall': 0.9781659388646288, 'f1-score': 0.9865426963542941, 'support': 2061}, 'rating': {'precision': 0.9914772727272727, 'recall': 0.9344042838018741, 'f1-score': 0.9620951068228808, 'support': 747}, 'feature request': {'precision': 0.9930693069306931, 'recall': 0.9794921875, 'f1-score': 0.9862340216322517, 'support': 1024}, 'micro avg': {'precision': 0.9945910861099091, 'recall': 0.9708553326293559, 'f1-score': 0.9825798867158276, 'support': 4735}, 'macro avg': {'precision': 0.9943357929514621, 'recall': 0.9666479391972183, 'f1-score': 0.9802165556421325, 'support': 4735}, 'weighted avg': {'precision': 0.9945757380741762, 'recall': 0.9708553326293559, 'f1-score': 0.9825144923570832, 'support': 4735}, 'samples avg': {'precision': 0.8463802083333334, 'recall': 0.83546875, 'f1-sc